<a href="https://colab.research.google.com/github/AShipulin/usurt/blob/main/%D0%9B%D0%9F%D0%A2%D0%A1_%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BB%D0%B0%D0%BD_%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B3%D0%BE_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%80%D0%B0_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp
import openpyxl
from openpyxl import load_workbook

def load_dynamic_data_from_excel(file_path):
    """Загружает данные из Excel файла с динамическими формулами"""
    # Открываем файл с openpyxl для вычисления формул
    wb = load_workbook(file_path, data_only=False)
    ws = wb['Решение']

    # Чтение стоимостей перевозок (с вычислением формул)
    costs = {}
    suppliers = [f'a{i+1}' for i in range(9)]
    consumers = [f'b{i+1}' for i in range(9)]

    for i, s in enumerate(suppliers):
        for j, c in enumerate(consumers):
            cell = ws.cell(row=5+i, column=3+j)
            costs[(s, c)] = cell.value if not str(cell.value).startswith('=') else cell.value

    # Чтение запасов поставщиков (с вычислением формул)
    supply = {}
    for i, s in enumerate(suppliers):
        cell = ws.cell(row=5+i, column=12)
        supply[s] = cell.value if not str(cell.value).startswith('=') else cell.value

    # Чтение спроса потребителей (с вычислением формул)
    demand = {}
    for j, c in enumerate(consumers):
        cell = ws.cell(row=14, column=3+j)
        demand[c] = cell.value if not str(cell.value).startswith('=') else cell.value

    # Преобразуем в DataFrame для удобства
    costs_df = pd.DataFrame(index=suppliers, columns=consumers)
    for (s, c), val in costs.items():
        costs_df.loc[s, c] = val

    return costs_df, supply, demand

def solve_dynamic_transportation(costs_df, supply, demand):
    """Решает транспортную задачу с динамическими данными"""
    # Создаем решатель
    solver = pywraplp.Solver.CreateSolver('SCIP')

    suppliers = list(supply.keys())
    consumers = list(demand.keys())

    # Создаем переменные
    x = {}
    for s in suppliers:
        for c in consumers:
            x[(s, c)] = solver.IntVar(0, solver.infinity(), f'x_{s}_{c}')

    # Ограничения по запасам
    for s in suppliers:
        if isinstance(supply[s], str) and supply[s].startswith('='):
            # Если значение - формула, устанавливаем большое верхнее ограничение
            solver.Add(sum(x[(s, c)] for c in consumers) <= 10000, f'Supply_{s}')
        else:
            solver.Add(sum(x[(s, c)] for c in consumers) <= supply[s], f'Supply_{s}')

    # Ограничения по спросу
    for c in consumers:
        if isinstance(demand[c], str) and demand[c].startswith('='):
            # Если значение - формула, устанавливаем большое нижнее ограничение
            solver.Add(sum(x[(s, c)] for s in suppliers) >= 0, f'Demand_{c}')
        else:
            solver.Add(sum(x[(s, c)] for s in suppliers) >= demand[c], f'Demand_{c}')

    # Целевая функция (только для числовых значений стоимости)
    total_cost = solver.Sum(
        x[(s, c)] * (costs_df.loc[s, c] if not isinstance(costs_df.loc[s, c], str) else 0)
        for s in suppliers
        for c in consumers
    )
    solver.Minimize(total_cost)

    # Решение задачи
    status = solver.Solve()

    # Анализ результатов
    if status == pywraplp.Solver.OPTIMAL:
        print('Оптимальное решение найдено!')
        print(f'Общая стоимость: {solver.Objective().Value():.2f}\n')

        # Создаем DataFrame для результатов
        results = pd.DataFrame(0, index=suppliers, columns=consumers)

        for s in suppliers:
            for c in consumers:
                val = x[(s, c)].solution_value()
                if val > 0:
                    results.loc[s, c] = val

        print("Оптимальный план перевозок (ненулевые значения):")
        print(results.replace(0, '').to_string())

        # Сохраняем результаты в Excel
        results.to_excel('dynamic_transportation_solution.xlsx')
        print("\nРезультаты сохранены в файл 'dynamic_transportation_solution.xlsx'")
    else:
        print('Решение не найдено или задача не имеет оптимального решения')

# Основная программа
if __name__ == "__main__":
    # Укажите путь к вашему файлу Excel
    excel_file = 'Часть II. Лекция. Траспортная задача. Большая размерность.xlsx'

    try:
        print("Загрузка данных из Excel...")
        costs_df, supply, demand = load_dynamic_data_from_excel(excel_file)

        print("\nМатрица стоимостей (первые 5x5):")
        print(costs_df.iloc[:5, :5].to_string())
        print("\nЗапасы поставщиков:")
        print(pd.Series(supply))
        print("\nСпрос потребителей:")
        print(pd.Series(demand))

        print("\nРешение задачи...")
        solve_dynamic_transportation(costs_df, supply, demand)

    except Exception as e:
        print(f"Произошла ошибка: {str(e)}")

Загрузка данных из Excel...

Матрица стоимостей (первые 5x5):
    b1  b2  b3  b4  b5
a1   7   9  21   7  19
a2  20  15  16  75  82
a3  84  38  62  12   4
a4  12  39  21  12   6
a5  88  18  94  25  72

Запасы поставщиков:
a1    238
a2    432
a3    359
a4    337
a5    390
a6    488
a7    446
a8    614
a9    367
dtype: int64

Спрос потребителей:
b1    482
b2    334
b3    414
b4    263
b5    380
b6    309
b7    571
b8    482
b9    436
dtype: int64

Решение задачи...
Оптимальное решение найдено!
Общая стоимость: 56065.00

Оптимальный план перевозок (ненулевые значения):
     b1   b2   b3   b4   b5   b6   b7   b8   b9
a1       238                                   
a2        96  336                              
a3                      318   41               
a4  197        78        62                    
a5                                     115  275
a6                                488          
a7  285                                     161
a8                 263       268   83       